In [1]:
import nltk
# Download stop words for Bahasa Indonesia (Jangan lupa jalankan sekali)
nltk.download('stopwords')
nltk.download('punkt')
# nltk.data.path.append("/root/nltk_data")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\herutriana44\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\herutriana44\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import re
import string
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('punkt')
nltk.download('wordnet')

def preprocessing_text(text):
    # Mengganti karakter newline dengan spasi
    text = re.sub(r'\n', ' ', text)

    # Menghapus tanda kurung buka dan tutup
    text = re.sub(r'\(', '', text)
    text = re.sub(r'\)', '', text)

    # Menghapus koma
    text = re.sub(r',', '', text)

    # Menghapus tanda hubung
    text = re.sub(r'-', '', text)

    # Menghapus tanda slash
    text = re.sub(r'/', '', text)

    # Menghapus tanda tanya
    text = re.sub(r'\?', '', text)

    # Menghapus karakter khusus
    text = re.sub(f"[{re.escape(string.punctuation)}]", '', text)

    # Menghapus URL
    text = re.sub(r'http\S+', '', text)

    # Mengubah teks menjadi huruf kecil (case folding)
    text = text.lower()

    # Tokenisasi menggunakan split
    words = text.split()  # Ini adalah tokenisasi sederhana, Anda dapat menggantinya dengan metode tokenisasi yang lebih canggih jika diperlukan

    # Menghapus stop words (menggunakan Sastrawi)
    stopword_factory = StopWordRemoverFactory()
    stop_words = stopword_factory.get_stop_words()  # Mengambil daftar stop words dalam bahasa Indonesia
    words = [word for word in words if word not in stop_words]

    # Stemming (menggunakan Sastrawi)
    stemmer = StemmerFactory().create_stemmer()
    text = ' '.join([stemmer.stem(word) for word in words])

    return text


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\herutriana44\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\herutriana44\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import pickle

# Load dataset
df = pd.read_csv('training.csv')

df['Questions'] = [preprocessing_text(str(df['Questions'][i])) for i in range(len(df))]

# Preprocess dataset
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['Questions'])
y = df['Answers']

# Train a classifier
classifier = MultinomialNB()
classifier.fit(X, y)

# Save the classifier and tfidf vectorizer to pickle files
with open('classifier_model.pkl', 'wb') as model_file:
    pickle.dump(classifier, model_file)

with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(tfidf_vectorizer, vectorizer_file)

# Function to get an answer
def get_answer(user_input):
    user_input_tfidf = tfidf_vectorizer.transform([user_input])
    answer = classifier.predict(user_input_tfidf)[0]
    return answer

# Test the chatbot
user_input = "bagaimana cara mendapatkan sptjm?"
answer = get_answer(user_input)
print("Jawaban:", answer)


Jawaban: Tentang SPTJM
1. Unduh Template (http://ringkas.kemdikbud.go.id/SPTJMMSIB5)
2. Isi dengan valid, Isi data diri, kondisi menerima beasiswa lain atau tidak, nama ortu dan tanda tangan, materai 10000 dan tanda tangan mahasiswa, dan nama Dekan
3. SPTJM ditanda tangani Dekan Fakultas, 
Dr. Mayasari, S.S., M.Hum
NIP. 197909262021212005
4. Unggah SPTJM, unggah scan SPTJM (dengan kondisi yang baik dan terang), sudah ttd materai dan ortu/wali, sudah ada nama dekan.
5. Folder SPTJM, unggah versi .docx ke folder https://bit.ly/uploadSPTJM_MSIB5
6. Unduh SPTJM, ada di folder https://bit.ly/unduhSPTJM_MSIB


In [4]:
df['Questions']

0                                               apa msib
1                              apa definisi istilah msib
2                            bagaimana jelas konsep msib
3                                apa maksud singkat msib
4                                   bisa beri jelas msib
                             ...                        
905    bagaimana pihak kampus institusi bantu mahasis...
906    dapat beri informasi berapa umum masalah ketid...
907    bagaimana tetap libat giat ekstrakurikuler org...
908    bagaimana pihak kampus institusi komunikasi ma...
909           beri prioritas terbit ktm aju mohon terima
Name: Questions, Length: 910, dtype: object

In [5]:
# Load validation dataset
df_validasi = pd.read_csv('validasi.csv')
X_validasi = [preprocessing_text(str(df_validasi['Questions'][i])) for i in range(len(df_validasi))]
y_validasi = df_validasi['Answers']


In [6]:
X_validasi_tfidf = tfidf_vectorizer.transform(X_validasi)


In [7]:
prediksi_validasi = classifier.predict(X_validasi_tfidf)


In [8]:
from sklearn.metrics import accuracy_score, classification_report

# Hitung akurasi
akurasi = accuracy_score(y_validasi, prediksi_validasi)
print("Akurasi:", akurasi)

# Tampilkan laporan klasifikasi yang mencakup precision, recall, dan f1-score
laporan_klasifikasi = classification_report(y_validasi, prediksi_validasi)
print("Laporan Klasifikasi:\n", laporan_klasifikasi)

Akurasi: 0.8700787401574803


c:\Users\herutriana44\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\herutriana44\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\herutriana44\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

Laporan Klasifikasi:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      precision    recall  f1-score   support

                                                                                                                                                                                                                                                                                                    

In [9]:
misclassified_examples = []
for i, (prediksi, jawaban_sebenarnya, pertanyaan) in enumerate(zip(prediksi_validasi, y_validasi, X_validasi)):
    if prediksi != jawaban_sebenarnya:
        misclassified_examples.append((i, prediksi, jawaban_sebenarnya, pertanyaan))

print("\nContoh Jawaban yang Salah:")
for i, prediksi, jawaban_sebenarnya, pertanyaan in misclassified_examples:
    print(f"Indeks: {i}, Prediksi: {prediksi}, Jawaban Sebenarnya: {jawaban_sebenarnya}")
    print(f"Pertanyaan: {pertanyaan}\n")


Contoh Jawaban yang Salah:
Indeks: 0, Prediksi: 1. 18 April - 7 Juni 2023 : Pendaftaran Mahasiswa
2. 8 - 18 Juni 2023 : Verifikasi dan validasi berkas mahasiswa
3. 21 - 23 Juni 2023 : Tes Kebinekaan
4. 1 - 30 Juli 2023 : Seleksi dan offering mahasiswa oleh mitra
5. Minggu ke-2 Agustus 2023 : Pengumuman hasil seleksi
6. 14 Agustus 2023 : National Onboarding, Jawaban Sebenarnya: Program Magang dan Studi Independen Bersertifikat Kampus Merdeka memberikan kesempatan kepada mahasiswa untuk mengasah dan mendapatkan kemampuan, pengetahuan dan sikap di dunia industri dengan cara bekerja dan belajar secara langsung dalam proyek atau permasalahan riil. Pengalaman belajar di luar perguruan tinggi selama lebih dari 16 minggu hingga 24 minggu dengan aktivitas pembelajaran yang terstruktur akan dikonversi ke SKS Mahasiswa
Pertanyaan: apa magang studi independen sertifikat

Indeks: 1, Prediksi: 1. 18 April - 7 Juni 2023 : Pendaftaran Mahasiswa
2. 8 - 18 Juni 2023 : Verifikasi dan validasi berkas mah

In [10]:
# Menghitung jumlah data yang benar
jumlah_benar = (y_validasi == prediksi_validasi).sum()

# Menghitung total jumlah data
total_data = len(y_validasi)

# Menampilkan jumlah data yang benar dan total jumlah data
print("Jumlah Data yang Benar:", jumlah_benar)
print("Total Data:", total_data)

Jumlah Data yang Benar: 221
Total Data: 254


In [12]:
import pickle

# Fungsi untuk memuat model dari file
def load_model(model_filename, vectorizer_filename):
    with open(model_filename, 'rb') as model_file:
        model = pickle.load(model_file)
    
    with open(vectorizer_filename, 'rb') as vectorizer_file:
        vectorizer = pickle.load(vectorizer_file)
    
    return model, vectorizer

# Fungsi untuk melakukan prediksi
def predict_answer(user_input, model, vectorizer):
    user_input_tfidf = vectorizer.transform([user_input])
    answer = model.predict(user_input_tfidf)[0]
    return answer

# Memuat model dan vectorizer
classifier_model, tfidf_vectorizer = load_model('classifier_model.pkl', 'tfidf_vectorizer.pkl')

# Fungsi untuk mendapatkan jawaban berdasarkan input pengguna
def get_answer_using_model(user_input):
    # Lakukan preprocessing pada input pengguna
    preprocessed_input = preprocessing_text(user_input)
    # Lakukan prediksi jawaban
    answer = predict_answer(preprocessed_input, classifier_model, tfidf_vectorizer)
    return answer

# Contoh penggunaan fungsi prediksi
user_input = "bagaimana cara mendapatkan sptjm?"
predicted_answer = get_answer_using_model(user_input)
print("Prediksi Jawaban:", predicted_answer)


Prediksi Jawaban: Tentang SPTJM
1. Unduh Template (http://ringkas.kemdikbud.go.id/SPTJMMSIB5)
2. Isi dengan valid, Isi data diri, kondisi menerima beasiswa lain atau tidak, nama ortu dan tanda tangan, materai 10000 dan tanda tangan mahasiswa, dan nama Dekan
3. SPTJM ditanda tangani Dekan Fakultas, 
Dr. Mayasari, S.S., M.Hum
NIP. 197909262021212005
4. Unggah SPTJM, unggah scan SPTJM (dengan kondisi yang baik dan terang), sudah ttd materai dan ortu/wali, sudah ada nama dekan.
5. Folder SPTJM, unggah versi .docx ke folder https://bit.ly/uploadSPTJM_MSIB5
6. Unduh SPTJM, ada di folder https://bit.ly/unduhSPTJM_MSIB


In [11]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# Load dataset
df = pd.read_csv('training_info.csv')

# Load BERT model and tokenizer
model_name = 'bert-base-multilingual-cased'  # You can choose a different pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Preprocess dataset
questions = df['Questions'].tolist()
answers = df['Answers'].tolist()

# Fine-tune BERT model if needed

# Create a text classification pipeline
nlp = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Function to get an answer
def get_answer(user_input):
    result = nlp(user_input)
    predicted_label = result[0]['label']
    return predicted_label

# Test the chatbot
user_input = "bagaimana cara mendapatkan sptjm?"
answer = get_answer(user_input)
print("Jawaban:", answer)

ModuleNotFoundError: No module named 'transformers'